# **Problem Statement**
## **Build a dog breed image classification model using Resnet50 CNN architecture.**

### **In this notebook I've used transfer learning and took pre-trained model from google's Tensorflow Hub and re-train that on Dog dataset. Using pre-trained model saves a lot of time and computational budget for new classification problem at hand.**

#### **Importing all necessary libraries**

In [76]:
import pandas as pd 
import numpy as np 

from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator

import warnings
warnings.filterwarnings("ignore")

#### **Loading the Dataset**

In [77]:
df=pd.read_csv('/content/drive/MyDrive/DL/labels.csv')
df.head() 

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


#### **Train Images**

In [78]:
train_path= r'/content/drive/MyDrive/DL/train/'

train_df=df.assign(img_path=lambda x: train_path + x['id'] +'.jpg')
print(train_df.shape)


(10222, 3)


In [79]:
train_df.head()

,id,breed,img_path
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,/content/drive/MyDrive/DL/train/000bec180eb18c...
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,/content/drive/MyDrive/DL/train/001513dfcb2ffa...
2,001cdf01b096e06d78e9e5112d419397,pekinese,/content/drive/MyDrive/DL/train/001cdf01b096e0...
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,/content/drive/MyDrive/DL/train/00214f311d5d22...
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/content/drive/MyDrive/DL/train/0021f9ceb3235e...


##### **Extracted only these mentioned breeds: beagle, chihuahua, doberman,french_bulldog, golden_retriever, malamute, pug, saint_bernard, scottish_deerhound,tibetan_mastiff.**


In [80]:
train_df1= train_df[(train_df.breed=='beagle') | (train_df.breed=='chihuahua') | (train_df.breed=='doberman') | 
             (train_df.breed=='french_bulldog') | (train_df.breed=='golden_retriever') | 
             (train_df.breed=='malamute') | (train_df.breed=='pug') | (train_df.breed=='saint_bernard') | 
             (train_df.breed=='scottish_deerhound') | (train_df.breed=='tibetan_mastiff')]
train_df1.head()

,id,breed,img_path
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,/content/drive/MyDrive/DL/train/0021f9ceb3235e...
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,/content/drive/MyDrive/DL/train/0042188c895a2f...
20,008b1271ed1addaccf93783b39deab45,doberman,/content/drive/MyDrive/DL/train/008b1271ed1add...
25,00a366d4b4a9bbb6c8a63126697b7656,golden_retriever,/content/drive/MyDrive/DL/train/00a366d4b4a9bb...
37,0100f55e4f0fe28f2c0465d3fc4b9897,golden_retriever,/content/drive/MyDrive/DL/train/0100f55e4f0fe2...


##### **Converted each image from 'train_df.img_path' into NumPy array for use with deep learning models using img_to_array() function.**

In [81]:
X=np.array([img_to_array(load_img(img, target_size=(224, 224))) for img in train_df1['img_path'].values.tolist()])
X.shape

(841, 224, 224, 3)

##### **Encoding Categorical Values in train_df1.breed**

In [82]:
train_df1.breed = train_df1.breed.astype('category')
train_df1.dtypes

id            object
breed       category
img_path      object
dtype: object

In [83]:
train_df1["breed_cat"] = train_df1.breed.cat.codes
train_df1.breed_cat.unique()

array([4, 8, 2, 5, 1, 6, 7, 0, 3, 9], dtype=int8)

In [84]:
y= train_df1.iloc[:,-1]  # Target column

In [85]:
X[0]

array([[[ 83.,  85.,  82.],
        [ 83.,  85.,  82.],
        [ 83.,  85.,  82.],
        ...,
        [ 86.,  86.,  84.],
        [ 78.,  84.,  80.],
        [ 78.,  84.,  80.]],

       [[ 78.,  80.,  77.],
        [ 78.,  80.,  77.],
        [ 78.,  80.,  77.],
        ...,
        [ 86.,  86.,  84.],
        [ 78.,  84.,  80.],
        [ 78.,  84.,  80.]],

       [[ 79.,  81.,  78.],
        [ 79.,  81.,  78.],
        [ 79.,  81.,  78.],
        ...,
        [ 86.,  86.,  84.],
        [ 79.,  84.,  80.],
        [ 79.,  84.,  80.]],

       ...,

       [[141., 141., 139.],
        [ 70.,  70.,  68.],
        [ 76.,  76.,  74.],
        ...,
        [ 85.,  87.,  84.],
        [ 80.,  82.,  77.],
        [ 89.,  91.,  86.]],

       [[ 76.,  78.,  75.],
        [ 87.,  89.,  86.],
        [ 91.,  93.,  90.],
        ...,
        [ 85.,  87.,  84.],
        [ 85.,  86.,  81.],
        [ 86.,  87.,  82.]],

       [[ 93.,  95.,  92.],
        [ 90.,  92.,  89.],
        [ 87.,  

##### **Preprocessing: scale images**

In [86]:
X= np.array(X)
X= X/255
y= np.array(y)

##### **Importing all necessary libraries for Neural network**

In [87]:
import tensorflow_hub
from tensorflow.keras import layers, models 

##### **Creating classifier using Resnet50.**

In [88]:
model = models.Sequential([
    tensorflow_hub.KerasLayer("https://tfhub.dev/tensorflow/resnet_50/feature_vector/1", input_shape=(224, 224, 3), trainable=False),
    layers.Dense(10,activation='softmax')
])


##### **Train test split**

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.28, random_state=6)

In [90]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 2048)              23561152  
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,581,642
Trainable params: 20,490
Non-trainable params: 23,561,152
_________________________________________________________________


In [91]:
model.compile(
  optimizer="adam",
  loss="sparse_categorical_crossentropy",
  metrics=['accuracy'])

##### **Train the Model**

In [92]:
model.fit(X_train, y_train, validation_split=0.35, batch_size = 10, epochs = 100)

Epoch 1/100
40/40 [==============================] - 9s 95ms/step - loss: 1.7270 - accuracy: 0.4644 - val_loss: 0.2233 - val_accuracy: 0.9528
Epoch 2/100
40/40 [==============================] - 3s 78ms/step - loss: 0.1495 - accuracy: 0.9759 - val_loss: 0.1315 - val_accuracy: 0.9764
Epoch 3/100
40/40 [==============================] - 3s 78ms/step - loss: 0.0604 - accuracy: 1.0000 - val_loss: 0.1100 - val_accuracy: 0.9811
Epoch 4/100
40/40 [==============================] - 3s 78ms/step - loss: 0.0281 - accuracy: 1.0000 - val_loss: 0.1032 - val_accuracy: 0.9717
Epoch 5/100
40/40 [==============================] - 3s 77ms/step - loss: 0.0213 - accuracy: 1.0000 - val_loss: 0.0921 - val_accuracy: 0.9811
Epoch 6/100
40/40 [==============================] - 3s 77ms/step - loss: 0.0202 - accuracy: 1.0000 - val_loss: 0.0854 - val_accuracy: 0.9811
Epoch 7/100
40/40 [==============================] - 3s 77ms/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.0837 - val_accuracy: 0.9764
Epoch 

In [93]:
model.evaluate(X_test,y_test)

8/8 [==============================] - 1s 129ms/step - loss: 0.0985 - accuracy: 0.9788


[0.09849816560745239, 0.9788135886192322]

##### **Making Prediction**

In [94]:
prediction= model.predict(X_test)
prediction[:5]

array([[1.81716605e-05, 3.95056441e-06, 6.14519536e-07, 3.44764516e-07,
        9.99895334e-01, 2.88215824e-06, 4.97287510e-06, 2.23245534e-05,
        1.76987869e-05, 3.37291713e-05],
       [9.99450147e-01, 1.51452841e-04, 1.71938973e-06, 5.09330948e-06,
        2.41722337e-05, 1.26367615e-06, 2.31540099e-07, 3.48651432e-04,
        1.72744058e-05, 2.36950122e-08],
       [1.97156478e-04, 8.91401360e-05, 9.61818223e-05, 3.96355790e-06,
        2.32230218e-06, 9.98851776e-01, 6.70679146e-05, 1.35368609e-05,
        1.02198108e-04, 5.76590886e-04],
       [5.00908093e-10, 3.62032937e-09, 1.00000000e+00, 5.60127944e-09,
        1.35026296e-10, 1.15096516e-10, 5.12816872e-11, 3.06524320e-11,
        2.76473973e-08, 8.93974117e-10],
       [4.23081365e-05, 3.17794786e-07, 9.33097333e-09, 1.31199798e-07,
        9.99956846e-01, 9.48811518e-09, 7.02759362e-09, 1.80223481e-07,
        8.17725834e-08, 3.18220272e-09]], dtype=float32)

In [95]:
predicted = [np.argmax(i) for i in prediction]
predicted[:5]

[4, 0, 5, 2, 4]

##### **Importing all necessary libraries for Evaluating the Model Performance**

In [96]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

##### **Classification Report**

In [97]:
target_names = ['beagle', 'chihuahua', 'doberman', 'french_bulldog', 'golden_retriever', 'malamute', 'pug', 'saint_bernard', 'scottish_deerhound', 'tibetan_mastiff']
print(classification_report(y_test, predicted, target_names=target_names))

                    precision    recall  f1-score   support

            beagle       1.00      1.00      1.00        35
         chihuahua       0.88      1.00      0.94        15
          doberman       1.00      0.94      0.97        18
    french_bulldog       0.94      0.94      0.94        17
  golden_retriever       0.95      0.95      0.95        21
          malamute       1.00      1.00      1.00        24
               pug       1.00      0.96      0.98        28
     saint_bernard       0.95      1.00      0.98        21
scottish_deerhound       1.00      1.00      1.00        32
   tibetan_mastiff       1.00      0.96      0.98        25

          accuracy                           0.98       236
         macro avg       0.97      0.98      0.97       236
      weighted avg       0.98      0.98      0.98       236



##### **Confusion Matrix**

In [98]:
print(confusion_matrix(y_test, predicted))

[[35  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  0  0  0  0]
 [ 0  1 17  0  0  0  0  0  0  0]
 [ 0  0  0 16  0  0  0  1  0  0]
 [ 0  1  0  0 20  0  0  0  0  0]
 [ 0  0  0  0  0 24  0  0  0  0]
 [ 0  0  0  1  0  0 27  0  0  0]
 [ 0  0  0  0  0  0  0 21  0  0]
 [ 0  0  0  0  0  0  0  0 32  0]
 [ 0  0  0  0  1  0  0  0  0 24]]


#####**Roc Auc_ Score**

In [99]:
print('roc_auc_score')
print(roc_auc_score(y_test, prediction, multi_class= 'ovo'))

roc_auc_score
0.999378305643245
